# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298522444760                   -0.85    5.2         
  2   -8.300206711527       -2.77       -1.25    1.0    127ms
  3   -8.300432881893       -3.65       -1.89    3.0    170ms
  4   -8.300462428558       -4.53       -2.76    2.2    154ms
  5   -8.300464135753       -5.77       -3.04    9.5    365ms
  6   -8.300464396202       -6.58       -3.22    2.2    158ms
  7   -8.300464521678       -6.90       -3.37    1.5    138ms
  8   -8.300464591619       -7.16       -3.54    1.4    145ms
  9   -8.300464630523       -7.41       -3.74    2.9    180ms
 10   -8.300464634828       -8.37       -3.82    1.0    133ms
 11   -8.300464642448       -8.12       -4.07    1.6    187ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67431225253                   -0.70    6.4         
  2   -16.67868832060       -2.36       -1.14    1.2    315ms
  3   -16.67920256700       -3.29       -1.87    2.4    340ms
  4   -16.67927828283       -4.12       -2.77    4.5    424ms
  5   -16.67928586964       -5.12       -3.17    4.6    438ms
  6   -16.67928618619       -6.50       -3.46    2.4    355ms
  7   -16.67928621536       -7.53       -3.93    1.6    306ms
  8   -16.67928622137       -8.22       -4.55    2.8    353ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32433266870                   -0.56    7.5         
  2   -33.33262623875       -2.08       -1.00    1.0    1.16s
  3   -33.33585173313       -2.49       -1.72    4.9    1.52s
  4   -33.33614980919       -3.53       -2.60    7.6    1.69s
  5   -33.33691744410       -3.11       -2.59    5.9    1.90s
  6   -33.33693190833       -4.84       -2.77    4.2    1.28s
  7   -33.33694333265       -4.94       -3.39    2.0    1.24s
  8   -33.33694376568       -6.36       -3.84    3.1    1.46s
  9   -33.33694377986       -7.85       -4.17    2.0    1.12s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298506472162                   -0.85    5.1         
  2   -8.300277989737       -2.75       -1.59    1.0   87.2ms
  3   -8.300433527221       -3.81       -2.61    3.4    163ms
  4   -8.300410763682   +   -4.64       -2.38    2.8    161ms
  5   -8.300464226347       -4.27       -3.33    1.1   93.5ms
  6   -8.300464586599       -6.44       -3.77    2.4    129ms
  7   -8.300464639190       -7.28       -4.24    1.8    108ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32728621392                   -0.56    7.6         
  2   -33.33302499477       -2.24       -1.28    1.2    1.01s
  3   -31.77370639573   +    0.19       -0.97    6.0    1.66s
  4   -32.80524309508        0.01       -1.20    5.0    1.68s
  5   -33.25605706866       -0.35       -1.37    3.4    1.52s
  6   -32.48201209986   +   -0.11       -1.10    4.1    1.56s
  7   -33.31924078502       -0.08       -1.84    3.9    1.37s
  8   -33.33509362177       -1.80       -2.23    2.0    1.17s
  9   -33.33612619373       -2.99       -2.52    2.8    1.19s
 10   -33.33684532852       -3.14       -2.77    2.9    1.31s
 11   -33.33691220012       -4.17       -3.01    2.8    1.19s
 12   -33.33693541177       -4.63      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.